In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
import osmnx as ox
import networkx as nx

In [2]:
! pip install osmnx

  Using cached https://files.pythonhosted.org/packages/69/77/98cbee7f94abe2c60013fd8751f2b2d6054d48078c61f9774cff0b341f43/osmnx-0.10-py2.py3-none-any.whl
    100% |████████████████████████████████| 1.8MB 9.8MB/s eta 0:00:01
    100% |████████████████████████████████| 15.0MB 2.7MB/s eta 0:00:01
    100% |████████████████████████████████| 10.1MB 4.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /Users/wangzixuan/Library/Caches/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [5]:
from IPython.display import IFrame 

In [6]:
import folium

In [7]:
from folium.plugins import HeatMap

In [8]:
import requests
import json

In [9]:
# the base URL
url = "https://www.rideindego.com/stations/json"
# website request has benn banned, get user agent in the network tap 
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
# get the response
r = requests.get(url,params={'format':'geojson'},headers=headers)

In [10]:
# convert to a GeoDataFrame
stations = gpd.GeoDataFrame.from_features(json.loads(r.text))

In [11]:
# remove missing
stations = stations.loc[stations.geometry.notnull()]
stations.head()

,addressCity,addressState,addressStreet,addressZipCode,bikesAvailable,classicBikesAvailable,closeTime,docksAvailable,electricBikesAvailable,eventEnd,...,latitude,longitude,name,notes,openTime,publicText,smartBikesAvailable,timeZone,totalDocks,trikesAvailable
0,Philadelphia,PA,1401 John F. Kennedy Blvd.,19102,3,3,03:55:00,27,0,None,...,39.95378,-75.16374,Municipal Services Building Plaza,None,04:00:00,,0,Eastern Standard Time,30,0
1,Philadelphia,PA,191 S. 2nd St.,19106,9,9,03:55:00,3,0,None,...,39.94733,-75.14403,"Welcome Park, NPS",None,04:00:00,,0,Eastern Standard Time,13,0
2,Philadelphia,PA,246 S. 40th St.,19104,2,2,03:55:00,15,0,None,...,39.95220,-75.20311,40th & Spruce,None,04:00:00,,0,Eastern Standard Time,17,0
3,Philadelphia,PA,328 S. 11th St.,19107,15,15,03:55:00,0,0,None,...,39.94517,-75.15993,"11th & Pine, Kahn Park",None,04:00:00,,0,Eastern Standard Time,15,0
4,Philadelphia,PA,1076 Berks Street,19122,5,5,03:55:00,13,0,None,...,39.98078,-75.15055,Temple University Station,None,04:00:00,,0,Eastern Standard Time,19,0


In [12]:
# make a NumPy array (use the "values" attribute)
coordinates = stations[['latitude', 'longitude']].values

In [13]:
coordinates[1]

array([ 39.94733, -75.14403])

In [14]:
# change the column class in order to cooporate with pop-up and color definition 
stations['classicBikesAvailable'] = stations['classicBikesAvailable'].astype(str) 
stations['bikesAvailable'] = stations['bikesAvailable'].astype(int) 

In [15]:
# define the icon color in terms of the number of available bikes 
# if there is less than 6 bikes available at the station, the icon would be red.
def color_producer(value):
    if value < 6:
        return 'red'
    else:
        return 'green'

In [16]:
from folium.plugins import MarkerCluster

In [17]:
# markers styled by the number of available bikes 
# use marker cluster function to identify the cluster areas
map2 = folium.Map(location=[39.95, -75.18], zoom_start=13)

marker_cluster = MarkerCluster().add_to(map2)

for point in range(0, len(coordinates)):
    folium.Marker(coordinates[point],
                  popup= folium.Popup('Available Bikes: '+ stations['classicBikesAvailable'][point], max_width=120),
                  icon=folium.Icon(color=color_producer(stations['bikesAvailable'][point]))
                 ).add_to(marker_cluster)
    
map2.save('indego_stations.html')

In [18]:
IFrame('indego_stations.html', width=900, height=600)

In [19]:
#heat map of the station data 
m = folium.Map(
    location=[39.95, -75.18],
    tiles='Cartodb Positron',
    zoom_start=13
)

# add heat map
HeatMap(coordinates).add_to(m)
m.save('heatmap_stations.html')

In [20]:
m